In [5]:
from SimpleHandEye.solvers import OpenCVSolver
from SimpleHandEye.interfaces.apriltag import ApriltagTracker
import pandas as pd
import numpy as np
import pinocchio as pin
import os 
import pickle

In [33]:
tag_size = 0.171
with open(os.path.join(dataset_path, 'camera_params.pkl'), 'rb') as f:
    int_params = pickle.load(f)
K = int_params['int_params']['RGB']['K']
tracker = ApriltagTracker(tag_size=tag_size, intrinsic_matrix=K)

In [7]:
poses = pd.read_csv(os.path.join(dataset_path, 'pose.csv'))
stamps = poses.loc[:,'id'].to_numpy()
trans = poses.loc[:,'x':'z'].to_numpy()
rots = poses.loc[:,'qx':'qw'].to_numpy()

In [24]:
import os
import cv2
dataset_path = 'dataset/apr24'
images = os.listdir(os.path.join(dataset_path, 'rgb'))
images.sort(key=lambda x:int(x.split('.')[0]))
imgs = []
poses = []
cam_Ts_tag = []
base_Ts_ef = []
tag_corners = []
for image in images:
    stamp = int(image.split('.')[0])
    img = cv2.imread(os.path.join(dataset_path,'rgb', image))
    idx = np.where(stamps.squeeze()==stamp)[0]
    if len(idx) !=0:
        idx = idx[0]
        imgs.append(img)
        info = tracker.getPoseAndCorners(img, tag_id=0)
        cam_T_tag = info['pose']
        corners = info['corners']
        tag_corners.append(corners)
        cam_Ts_tag.append(cam_T_tag)
        R = pin.Quaternion(rots[idx,:]).matrix()
        t = trans[idx,:].reshape(3,1)
        base_T_ef = np.hstack([R,t])
        base_T_ef = np.vstack([base_T_ef, np.array([0,0,0,1])])
        base_Ts_ef.append(base_T_ef)

In [25]:
A = base_Ts_ef
B = cam_Ts_tag
solver = OpenCVSolver(type='AX=YB')
X,Y = solver.solve(A, B)

In [26]:
base_T_cam = Y

In [57]:
base_T_cam

array([[ 0.40889382,  0.43091743, -0.80443521,  0.77993618],
       [ 0.90798509, -0.28047074,  0.31128643, -0.00220407],
       [-0.09148179, -0.85769828, -0.50594935,  0.31299386],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [58]:
for i in range(len(imgs)):
    print((A[i]@X@np.linalg.inv(Y@B[i]))[:3,-1])

[ 0.00246872  0.01330266 -0.00407286]
[ 0.00572929 -0.00380917 -0.00100059]
[-0.00361441  0.02094144 -0.01737711]
[ 0.00188232 -0.01247749  0.0039812 ]
[ 0.00185214 -0.01214781  0.00417629]
[-0.00603116 -0.00993642  0.00769639]
[-0.0085883  -0.00264896  0.00409065]


In [50]:
pix_ps = np.vstack(tag_corners)[...,np.newaxis].tolist()

In [51]:
tag_ps = np.vstack(len(A)*[[np.array([[-tag_size/2, tag_size/2, 0],
                                [ tag_size/2, tag_size/2, 0],
                                [ tag_size/2, -tag_size/2, 0],
                                [-tag_size/2,  -tag_size/2, 0]])]]).tolist()

In [52]:
from SimpleHandEye.optimizers import AXYBReprojectionOptimizer
solver = AXYBReprojectionOptimizer(camera_T_tag='B', camera_matrix=K) 

In [53]:
solver.solve(A=A, B=B, X=X, Y=Y, pix_ps=pix_ps, tag_ps=tag_ps)

AssertionError: Gave args ([701.813720703125],) for <class 'symforce.geo.matrix.Matrix21'>